# Challenge StartUp.ML
# Airline On-Time Arrivals - Binary classification model
## Pierre Megret

### Main Goal

To use the US Dept. of Transportation on-time arrival data for non-stop domestic flights by major air carriers to predict arrival delays with a binary classification model.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
from sklearn.externals import joblib
import warnings
warnings.filterwarnings("ignore") # In Pandas, convert_objects is deprecated.

### Importing the data set

In [2]:
data = pd.read_csv('Jan-2016-Class.csv')

In [3]:
data.head()

,DAY_OF_WEEK,UNIQUE_CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DEL15,DIVERTED,AIR_TIME,DISTANCE,Unnamed: 13
0,1,F9,141,11292,30325,14747,30559,1645,1845,0.0,0.0,140.0,1024.0,NaN
1,1,F9,142,14747,30559,11292,30325,1930,2310,0.0,0.0,131.0,1024.0,NaN
2,1,F9,402,12892,32575,11292,30325,1810,2126,0.0,0.0,113.0,862.0,NaN
3,1,F9,612,11292,30325,13303,32467,2255,433,0.0,0.0,201.0,1709.0,NaN
4,1,F9,1152,10397,30397,13204,31454,840,1005,0.0,0.0,62.0,404.0,NaN


##### The data set
The data set corresponds of the month of January 2016, so that I could test my model on all the year.

To predict whether a flight will be delayed or not, here's the features I chose from the USDoT:
1. Day of Week: there might be more delayed on the weekend, if more people take the planes;
2. Airline Id: some airlines have probably more delays than others;
3. Flight Number: some flights might be more likely to have delays than others;
4. Origin and Destination Airport: some airport might be more crowded than others;
5. Origin and Destination Cities: some cities might have poor airport(s);
6. CRS departure and arrival time: there might be more delays in the morning than in the afternoon;
7. ARR_DEL15: a dummy variable for delay, 1 if the flight has been delayed, 0 otherwise;
8. DIVERTED: a dummy variable to know whether a flight has been diverted (1) or no (0);
9. Air Time: time, in minutes, that the plane has been in the air;
10. Distance: the distance between the two airports, in miles.

### Data Wrangling

In [4]:
print data.shape
print data.dtypes

(445827, 14)
DAY_OF_WEEK                int64
UNIQUE_CARRIER            object
FL_NUM                     int64
ORIGIN_AIRPORT_ID          int64
ORIGIN_CITY_MARKET_ID      int64
DEST_AIRPORT_ID            int64
DEST_CITY_MARKET_ID        int64
CRS_DEP_TIME               int64
CRS_ARR_TIME               int64
ARR_DEL15                float64
DIVERTED                 float64
AIR_TIME                 float64
DISTANCE                 float64
Unnamed: 13              float64
dtype: object


In [5]:
data = data.drop('Unnamed: 13',axis = 1) # Useless column
data.isnull().sum()

DAY_OF_WEEK                  0
UNIQUE_CARRIER               0
FL_NUM                       0
ORIGIN_AIRPORT_ID            0
ORIGIN_CITY_MARKET_ID        0
DEST_AIRPORT_ID              0
DEST_CITY_MARKET_ID          0
CRS_DEP_TIME                 0
CRS_ARR_TIME                 0
ARR_DEL15                12529
DIVERTED                     0
AIR_TIME                 12529
DISTANCE                     0
dtype: int64

In [6]:
for i in ['ARR_DEL15','AIR_TIME']:
    data = data.dropna(subset=[i]) # Getting rid of missing values, they are only 2% of all rows.

In [7]:
hour_func = lambda x: int(str(x)[:2]) if len(str(x)) == 6 else int(str(x)[0])
data['DEP_HOUR'] = data['CRS_DEP_TIME'].apply(hour_func) # Taking only the hours as indicator of time, converting into integers
data['ARR_HOUR'] = data['CRS_ARR_TIME'].apply(hour_func)

In [8]:
data['ARR_HOUR'].head()

0    1
1    2
2    2
3    4
4    1
Name: ARR_HOUR, dtype: int64

In [9]:
print len(set(data['UNIQUE_CARRIER'])) # How many unique Airline Id?
print len(set(data['DAY_OF_WEEK']))    # Is there actually 7 days in this data set?
print len(set(data['FL_NUM']))         # How many different flight number exist in the data?

12
7
6654


In [10]:
print set(data['ORIGIN_AIRPORT_ID']) == set(data['DEST_AIRPORT_ID'])
print set(data['ORIGIN_CITY_MARKET_ID']) == set(data['DEST_CITY_MARKET_ID'])

True
True


In [11]:
# As the set of Destination and Origin airport and city are the same, printing the number of unique destination 
# airports and cities is enough.
print len(set(data['DEST_AIRPORT_ID']))
print len(set(data['DEST_CITY_MARKET_ID']))

294
273


In [12]:
len(data['DIVERTED'])

433298

In [13]:
len(data['ARR_DEL15'])

433298

In [14]:
f = pd.read_csv('Jan-2017-Class.csv') # How does the identification number assigned by US DOT looks like?
set(f['UNIQUE_CARRIER'])

{'AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'NK', 'OO', 'UA', 'VX', 'WN'}

In [15]:
# Converting the Airline Ids into integers, to be able to apply a scikit-learn binary classifier

Airlines_ID = {"AA":0 ,"AS" :1,"B6":2,"DL":3,"EV":4,"F9":5,"HA":6,"NK":7,"OO":8,"UA":9,"VX":10,"WN":11}

data['AIRLINE'] = data['UNIQUE_CARRIER'].replace(Airlines_ID)
data['AIRLINE'] = data['AIRLINE'].convert_objects(convert_numeric=True)

In [16]:
set(data['AIRLINE'])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

### Building the training and test sets

We use the dummy column ARR_DEL15 as our target, and the rest of the features as our parameters.

In [17]:
features = ['DAY_OF_WEEK','FL_NUM','ORIGIN_AIRPORT_ID','ORIGIN_CITY_MARKET_ID',
            'DEST_AIRPORT_ID','DEST_CITY_MARKET_ID','DIVERTED','AIR_TIME','DISTANCE','DEP_HOUR','ARR_HOUR','AIRLINE']
X = data[features].values
Y = data['ARR_DEL15'].values

In [18]:
print X.shape
print Y.shape

(433298L, 12L)
(433298L,)


In [19]:
print "There is %s of delayed flights and %s on on-time ones."%(len(Y[Y == 1]), len(Y[Y == 0]))

There is 70882 of delayed flights and 362416 on on-time ones.


In [20]:
X_train = X[:400645]
Y_train = Y[:400645]
                      # Creating a training and testing set from the data of January 2016
X_test = X[400645:]
Y_test = Y[400645:]

In [21]:
print "There is %s of delayed flights and %s on on-time ones, in the training set."%(len(Y_train[Y_train == 1]), len(Y_train[Y_train == 0]))
print "There is %s of delayed flights and %s on on-time ones, in the test set."%(len(Y_test[Y_test == 1]), len(Y_test[Y_test == 0]))

There is 65313 of delayed flights and 335332 on on-time ones, in the training set.
There is 5569 of delayed flights and 27084 on on-time ones, in the test set.


### Building the Model : Random Forest Classifier

 I will use the open source Python library Scikit-learn to implement a Random Forest Classifier.
##### Why Random Forest?
1. Usually RF algorithm does not overfit;
2. It provides excellent accuracy among current classification and regression algorithms;
3. It can be applied efficiently to large-scale datasets;
4. It handles high dimensional dataset;
5. It determines the most important features of a dataset;
6. It is easily interpretable.

I am using a k-fold cross-validation to determine the best number of trees the random forest classifier should have.

In [22]:
parameters = {'n_estimators':np.arange(100,130,5)}

In [23]:
Model = RandomForestClassifier()                               # Create the Model
cross_validation = grid_search.GridSearchCV(Model, parameters) # Define the Cross-Validation
cross_validation.fit(X_train, Y_train)                         # Run the Cross-Validation

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([100, 105, 110, 115, 120, 125])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [24]:
print cross_validation.best_params_
print cross_validation.best_score_

{'n_estimators': 115}
0.781706997467


In [25]:
Model = cross_validation.best_estimator_

In [ ]:
# Save the model to disk (cleared the output for visibility)
filename = 'model_binclass.sav'
joblib.dump(model, filename)

In [24]:
# Load the model
model = joblib.load('model_binclass.sav')
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=115, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
model.score(X_train, Y_train)

0.98434025134470671

In [26]:
model.score(X_train[10000:11790], Y_train[10000:11790])

0.9826815642458101

In [27]:
# Testing the model on the test set
# The higher the score, the higher the accuracy

predictions = model.predict(X_test)
score = 0
for i in range(len(Y_test)):
    if Y_test[i] == predictions[i]:
        score += 1
100*score / len(Y_test)

81

In [29]:
model.feature_importances_

array([ 0.15316854,  0.13358598,  0.03559167,  0.03330273,  0.03340057,
        0.03297339,  0.        ,  0.46093092,  0.05431568,  0.02001562,
        0.01792382,  0.02479109])

##### Most important features are:
1. Diverted
2. Day of Week
3. Flight number

## January 2017

Testing the model with the same month, but a year later.

In [30]:
data_test2 = pd.read_csv('Jan-2017-Class.csv')

In [31]:
data_test2 = data_test2.drop('Unnamed: 13',axis = 1)
for i in ['ARR_DEL15','AIR_TIME']:
    data_test2 = data_test2.dropna(subset=[i])
data_test2['DEP_HOUR'] = data_test2['CRS_DEP_TIME'].apply(hour_func)
data_test2['ARR_HOUR'] = data_test2['CRS_ARR_TIME'].apply(hour_func)
data_test2['AIRLINE'] = data_test2['UNIQUE_CARRIER'].replace(Airlines_ID)
data_test2['AIRLINE'] = data_test2['AIRLINE'].convert_objects(convert_numeric=True)

In [32]:
X_test2 = data_test2[features].values
Y_test2 = data_test2['ARR_DEL15'].values

In [33]:
predictions2 = model.predict(X_test2)
score2 = 0
for i in range(len(Y_test2)):
    if Y_test2[i] == predictions2[i]:
        score2 += 1
100 *score2 / len(Y_test2)

75

## February 2016

Testing the model with the month right after the training one.

In [34]:
data_test3= pd.read_csv('Feb-2016-Class.csv')

In [35]:
data_test3 = data_test3.drop('Unnamed: 13',axis = 1)
for i in ['ARR_DEL15','AIR_TIME']:
    data_test3 = data_test3.dropna(subset=[i])
data_test3['DEP_HOUR'] = data_test3['CRS_DEP_TIME'].apply(hour_func)
data_test3['ARR_HOUR'] = data_test3['CRS_ARR_TIME'].apply(hour_func)
data_test3['AIRLINE'] = data_test3['UNIQUE_CARRIER'].replace(Airlines_ID)
data_test3['AIRLINE'] = data_test3['AIRLINE'].convert_objects(convert_numeric=True)

In [36]:
X_test3 = data_test3[features].values
Y_test3 = data_test3['ARR_DEL15'].values

In [37]:
predictions3 = model.predict(X_test3)
score3 = 0
for i in range(len(Y_test3)):
    if Y_test3[i] == predictions3[i]:
        score3 += 1
100 *score3 / len(Y_test3)

80

### October 2016

Testing the model with a month in the same year as the training set.

In [38]:
data_test4= pd.read_csv('Oct-2016-Class.csv')

In [39]:
data_test4.columns

Index([u'DAY_OF_WEEK', u'UNIQUE_CARRIER', u'FL_NUM', u'ORIGIN_AIRPORT_ID',
       u'ORIGIN_CITY_MARKET_ID', u'DEST_AIRPORT_ID', u'DEST_CITY_MARKET_ID',
       u'CRS_DEP_TIME', u'CRS_ARR_TIME', u'ARR_DEL15', u'DIVERTED',
       u'AIR_TIME', u'DISTANCE', u'Unnamed: 13'],
      dtype='object')

In [40]:
data_test4 = data_test4.drop('Unnamed: 13',axis = 1)
for i in ['ARR_DEL15','AIR_TIME']:
    data_test4 = data_test4.dropna(subset=[i])
data_test4['DEP_HOUR'] = data_test4['CRS_DEP_TIME'].apply(hour_func)
data_test4['ARR_HOUR'] = data_test4['CRS_ARR_TIME'].apply(hour_func)
data_test4['AIRLINE'] = data_test4['UNIQUE_CARRIER'].replace(Airlines_ID)
data_test4['AIRLINE'] = data_test4['AIRLINE'].convert_objects(convert_numeric=True)

In [41]:
X_test4 = data_test4[features].values
Y_test4 = data_test4['ARR_DEL15'].values

In [42]:
predictions4 = model.predict(X_test4)
score4 = 0
for i in range(len(Y_test4)):
    if Y_test4[i] == predictions4[i]:
        score4 += 1
100 *score4 / len(Y_test4)

80

### Conclusion

In [43]:
(81+75+80+80)/4

79

#### The binary classifier has an average accuracy of 79%